In [6]:
import urllib
from bs4 import BeautifulSoup
import requests
import numpy
import os
import re
import json
import unicodedata

In [2]:
url_general = 'https://www.factcheck.org/the-factcheck-wire/'
html_general = requests.get(url_general).text
soup_general = BeautifulSoup(html_general, 'html.parser')

In [3]:
num_pags = int(soup_general.find_all('span', attrs={'class' : 'sr-only'})[1].text)

print(num_pags)

328


In [4]:
def replace_characters(text):
    text_aux = ''.join(c for c in unicodedata.normalize('NFD', text.strip())
                  if unicodedata.category(c) != 'Mn')
    return re.sub(r'[^\w\s]', '', text_aux.lower().replace(u'\xa0', u' ').replace(u'\n', u' ')).split(" ")

In [9]:
bulos = {}

cont = 0
for i in range(1, num_pags+1):
    url = 'https://www.factcheck.org/the-factcheck-wire/page/' + str(i)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    for h3 in soup.find_all('h3',attrs={'class' : 'entry-title'}): 
        a = h3('a')[0]
        title = a.text
        link = a.get('href', None)
        words = dict()
        
        html_new = requests.get(link, allow_redirects=False).text
        
        soup_new = BeautifulSoup(html_new, 'html.parser')
        title_list = re.sub(r'[^\w\s]', '', title.lower()).split(" ")
        
        for w in title_list:
            if w in words:
                words[w] += 1
            else:
                words[w] = 1
                
        new_text = soup_new.find('div',attrs={'class' : "entry-content"})
        
        for text in new_text('p'):
            for w in replace_characters(text.get_text):
                if w in words:
                    words[w] += 1
                else:
                    words[w] = 1
        
        bulos[cont] = {"titulo" : title, "link" : link, "words_count" : words}
        cont += 1

In [10]:
bulos

{0: {'titulo': 'Viral Story Takes Fauci COVID-19 Vaccine Safety Comments Out of Context',
  'link': 'https://www.factcheck.org/2021/12/scicheck-viral-story-takes-fauci-covid-19-vaccine-safety-comments-out-of-context/',
  'words_count': {'viral': 2,
   'story': 5,
   'takes': 2,
   'fauci': 19,
   'covid19': 24,
   'vaccine': 35,
   'safety': 21,
   'comments': 2,
   'out': 10,
   'of': 83,
   'context': 5,
   'the': 146,
   'vaccines': 36,
   'available': 2,
   'in': 41,
   'us': 6,
   'were': 10,
   'found': 3,
   'to': 54,
   'be': 17,
   'safe': 8,
   'and': 66,
   'effective': 4,
   'clinical': 7,
   'trials': 10,
   'realworld': 1,
   'conditions': 1,
   'dr': 7,
   'anthony': 4,
   'did': 1,
   'not': 20,
   'admit': 1,
   'that': 38,
   'covid': 5,
   'may': 6,
   'actually': 10,
   'make': 11,
   'people': 20,
   'worse': 12,
   'as': 22,
   'a': 45,
   'headline': 2,
   'misleadingly': 1,
   'claims': 4,
   'faucis': 3,
   'march': 4,
   '2020': 6,
   'remarks': 3,
   'about':

In [6]:
len(bulos)

3264

In [9]:
with open('fakenewsFactCheck.json', 'w') as fp:
    json.dump(bulos, fp)